<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex11notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex11notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




板書や口頭で補足する前提なので，この notebook だけでは説明が不完全です．


---
## 混合正規分布モデルとEMアルゴリズム
---


---
### 準備


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture

In [ ]:
# NumPy の 疑似乱数生成器（rng = random number generator）
from numpy.random import default_rng
rng = default_rng() # 疑似乱数生成器を初期化

---
### 混合正規分布モデル

#### 単一の正規分布ではうまく表せそうにないデータ

The Old Faithful Geyser Dataset という2次元のデータ．
これは，米国イエローストーン国立公園内にある，「[The Old Faithful Geyser](https://ja.wikipedia.org/wiki/%E3%82%AA%E3%83%BC%E3%83%AB%E3%83%89%E3%83%BB%E3%83%95%E3%82%A7%E3%82%A4%E3%82%B9%E3%83%95%E3%83%AB%E3%83%BB%E3%82%AC%E3%82%A4%E3%82%B6%E3%83%BC)」
という間欠泉（一定周期で熱湯の噴出と停止を繰り返す）を観測して得られたデータ．

ここでは，以下の URL から取得したものを用いる．

https://gist.github.com/curran/4b59d1046d9e66f2787780ad51a1cd87




In [ ]:
URL = 'https://gist.github.com/curran/4b59d1046d9e66f2787780ad51a1cd87/raw/9ec906b78a98cf300947a37b56cfe70d01183200/data.tsv'
dfOFG = pd.read_csv(URL, sep='\t')
XOFG = dfOFG.to_numpy()
print(XOFG.shape)
dfOFG

eruptions は噴出の持続時間[分]，waiting は次の噴出までの時間[分]．
散布図は次の通り．

In [ ]:
# グラフを描く
fig, ax = plt.subplots()
ax.scatter(dfOFG['eruptions'], dfOFG['waiting'])
xmin, xmax = 0, 6
ymin, ymax = 40, 100
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_xlabel('Eruptions [minutes]')
ax.set_ylabel('Waiting [minutes]')
plt.show()

この2次元のデータに一つの正規分布を当てはめるのはよい考えとは言えなさそうである．
この例に限らず，世の中には単一の正規分布ではうまくモデル化できない．どうしよう？

#### 混合正規分布モデル

**混合正規分布モデル** (Gaussian Mixture Model, GMM) は，複数の多次元正規分布を組み合わせて複雑な分布を表現するものである．

$K$ 個の正規分布 ${\cal N}(\mathbf{x}; \mathbf{\mu}_k, \Sigma_k)$ を用いて，個々のデータが次のようにして生成されるとする:

1. 確率 $w_k$ で $k$ 番目の正規分布が選ばれる． $0 \leq w_k \leq 1$ ($k = 1, 2, \ldots, K$) かつ $\sum_{k=1}^{K}w_k = 1$ とする．
1. その正規分布に従ってデータが生成される



GMM を式で表そう．
まず，どの正規分布を選ぶかを表す確率変数 $z$ を導入する．
$z$ は $1, 2, \ldots, K$ のいずれかをとる離散確率変数である．
すると，

$$
\begin{aligned}
p(z = k) &= w_k\\
p(\mathbf{x}|z = k) &= {\cal N}(\mathbf{x}; \mathbf{\mu}_k, \Sigma_k)
\end{aligned}
$$

と表せる．このとき，

$$
\begin{aligned}
p(\mathbf{x}) = \sum_{k=1}^{K} p(\mathbf{x}|z = k) p(z = k)
\end{aligned}
$$

なので，$p(\mathbf{x})$ は次式のように表される．

$$
\begin{aligned}
p(\mathbf{x}) = \sum_{k=1}^{K} w_k {\cal N}(\mathbf{x}; \mathbf{\mu}_k, \Sigma_k)
\end{aligned}
$$

これが GMM の式である．
GMM のパラメータは，$w_k, \mathbf{\mu}_k, \Sigma_k$ ($k = 1, 2, \ldots, K$) である．



上記の導出過程では， $z$ という確率変数が登場した．
確率変数 $\mathbf{x}$ が観測できる（データが直接手に入る）のに対して，この確率変数 $z$ は観測できるものではない．
GMM において，データの生成過程に隠れて存在していると仮定した変数である．
このような変数のことを **潜在変数** (latent variable) という．
GMM は，潜在変数を持つ確率モデルの代表例である．


#### 実験: Old Faithful Dataset に GMM を当てはめてみる

GMM のパラメータをどうやって推定するかという話は後回しにして， The Old Faithful Dataset Geyser Dataset に GMM を当てはめてみよう．
ここでは， [sklearn.mixture.GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) を用いる．

In [ ]:
# GMM の当てはめ
K = 2
gmm = GaussianMixture(n_components=K)
gmm.fit(XOFG)

In [ ]:
# 推定されたモデルパラメータの表示
for k in range(K):
    print(f'### {k}-th component')
    print('weight = ', gmm.weights_[k])
    print('mu = ', gmm.means_[k])
    print('cov = ')
    print(gmm.covariances_[k])
    print()

In [ ]:
## データに正規分布を重ねて表示

# グラフ描画用のグリッドデータの作成
x_mesh, y_mesh = np.mgrid[xmin:xmax:(xmax-xmin)/100, ymin:ymax:(ymax-ymin)/100]
X_mesh = np.dstack((x_mesh, y_mesh))

# グラフ
fig, ax = plt.subplots()
ax.scatter(XOFG[:, 0], XOFG[:, 1])

# Gaussian を当てはめた結果
for k in range(K):
    mu = gmm.means_[k]
    cov = gmm.covariances_[k]
    ax.contour(x_mesh, y_mesh, multivariate_normal.pdf(X_mesh, mean=mu, cov=cov))
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

データの生成過程をモデル化しているので，疑似乱数を使ってこの GMM が表す分布に従うサンプルを無限に作り出すことができる．

In [ ]:
# データを生成
N = 200
z = rng.choice(K, size=N, p=gmm.weights_)
print(z)

Xgen = np.empty((N, 2))
for k in range(K):
    Nk = np.sum(z == k)
    mu = gmm.means_[k]
    cov = gmm.covariances_[k]
    Xgen[z == k, :] = rng.multivariate_normal(mu, cov, size=Nk)

# 最初の10個
for n in range(10):
    print(z[n], Xgen[n, :])

In [ ]:
# グラフを描く
fig, ax = plt.subplots(1, 2, figsize=(10, 4.5))
ax[0].scatter(XOFG[:, 0], XOFG[:, 1], label='observed data')
ax[0].scatter(Xgen[:, 0], Xgen[:, 1], label='generated')
ax[0].set_xlim(xmin, xmax)
ax[0].set_ylim(ymin, ymax)
ax[0].legend()
ax[1].scatter(Xgen[z == 0, 0], Xgen[z == 0, 1], label='k = 0', color='g')
ax[1].scatter(Xgen[z == 1, 0], Xgen[z == 1, 1], label='k = 1', color='m')
ax[1].set_xlim(xmin, xmax)
ax[1].set_ylim(ymin, ymax)
ax[1].legend()
plt.show()

左は，GMMを使って生成したデータを元のデータと重ねて描いたもの．
右は，その生成されたデータがどちらの正規分布から生成されたのかを色分けして描いたもの．

---
### EMアルゴリズムによる GMM のパラメータ推定

データ $\{ \mathbf{x}_n \in \mathbb{R}^{D} \}_{n=1}^{N}$ に GMM を当てはめる問題，すなわち，これらのデータに当てはまる GMM のパラメータを推定する問題を考える．

以下，GMM のパラメータをまとめて $\mathbf{\theta}$ という記号で表すことにすると，対数尤度 $L(\mathbf{\theta})$ は次のようになる．

$$
\begin{aligned}
L(\mathbf{\theta}) &= \log \left( \prod_{n=1}^N p_{\mathbf{\theta}}(\mathbf{x}_n) \right) \\
&= \sum_{n=1}^N \log{ p_{\mathbf{\theta}}(\mathbf{x}_n) } \\
&= \sum_{n} \log \left( \sum_{z_n} p_{\mathbf{\theta}}(\mathbf{x}_n, z_n)  \right)
\end{aligned}
$$

単一の正規分布の場合のようにこの対数尤度の最大化（最尤推定）を考えたいが，実はこの式のように $\log$ の中に $\sum$ が入った形をしているものは解析的に解くのが難しい．

#### 対数尤度 = ELBO + KL-divergence

対数尤度 $L(\mathbf{\theta})$ の最大化を実現する方法を探るために，式を変形してみる．
ここでは，簡単のため，ひとつのデータの対数尤度 $\log p_{\mathbf{\theta}}(\mathbf{x})$ について考える（添字 $n$ も省略）．

任意の確率分布 $q(z)$ に対して，$\log p_{\mathbf{\theta}}(\mathbf{x})$ は次のように式変形できる．

$$
\begin{aligned}
\log{ p_{\mathbf{\theta}}(\mathbf{x}) } &= \log{ p_{\mathbf{\theta}}(\mathbf{x}) } \sum_z q(z) \qquad  \because \sum_z q(z) = 1 \\
&= \sum_z q(z) \log p_{\mathbf{\theta}}(\mathbf{x}) \\
&= \sum_z q(z) \log \frac{p_{\mathbf{\theta}}(\mathbf{x}, z)}{p_{\mathbf{\theta}}(z|\mathbf{x})} \qquad  \because p_{\mathbf{\theta}}(\mathbf{x}, z) = p_{\mathbf{\theta}}(z|\mathbf{x})p_{\mathbf{\theta}}(\mathbf{x}) \\
&= \sum_z q(z) \log \left( \frac{p_{\mathbf{\theta}}(\mathbf{x}, z)}{p_{\mathbf{\theta}}(z|\mathbf{x})} \cdot \frac{q(z)}{q(z)} \right) \\
&= \underbrace{\sum_z q(z) \log \frac{p_{\mathbf{\theta}}(\mathbf{x}, z)}{q(z)}}_{{\rm ELBO}(\mathbf{x}; \mathbf{\theta}, q)} + \underbrace{\sum_z q(z) \log \frac{q(z)}{p_{\mathbf{\theta}}(z|\mathbf{x})}}_{D_{\rm KL}(q(z) \Vert p_{\mathbf{\theta}}(z|\mathbf{x}))}\\
&= {\rm ELBO}(\mathbf{x}; \mathbf{\theta}, q) + D_{\rm KL}(q(z) \Vert p_{\mathbf{\theta}}(z|\mathbf{x}))
\end{aligned}
$$

ここで，最後の式の第2項は，「$q(z)$ の $p_{\mathbf{\theta}}(z|\mathbf{x})$ に対するKLダイバージェンス」と呼ばれる量で，$0$ 以上の実数値をとる．
そのため，
$\log p_{\mathbf{\theta}}(\mathbf{x}) \geq {\rm ELBO}(\mathbf{x}; \mathbf{\theta}, q)$
が成り立つ．
すなわち，${\rm ELBO}(\mathbf{x}; \mathbf{\theta}, q)$ は $\log p_{\mathbf{\theta}}(\mathbf{x})$ の下界である（注）．
したがって，$q(z)$を適当に定めたうえで ${\rm ELBO}(\mathbf{x}; \mathbf{\theta}, q)$ を大きくすれば，$\log p_{\mathbf{\theta}}(\mathbf{x})$ も大きくなる．
また，その式は $\log p_{\mathbf{\theta}}(\mathbf{x})$ よりも扱いやすい形をしている．

※ ELBO: Evidence Lower BOund, エビデンスの下界







#### KL-divergence

**KLダイバージェンス** （KL情報量， Kullback-Leibler divergence）は，2つの確率分布の「遠さ」を表す指標．


連続確率分布 $p(x), q(x)$ において，$p(x)$ の $q(x)$ に対する KL ダイバージェンスは

$$
D_{\rm KL}(p \Vert q) = \int_{-\infty}^{\infty} p(x) \log \frac{p(x)}{q(x)} dx
$$

と定義される．また，離散確率分布 $p(x), q(x)$ において，$p(x)$ の $q(x)$ に対する KL ダイバージェンスは

$$
D_{\rm KL}(p \Vert q) = \sum_{x} p(x) \log \frac{p(x)}{q(x)}
$$

と定義される．

次の性質がある．

- $D_{\rm KL}(p \Vert q) \geq 0$．等号が成り立つのは2つの分布が一致する場合かつその場合に限られる
- 一般に $D_{\rm KL}(p \Vert q) \ne D_{\rm KL}(q \Vert p)$ となる．

直感的には，2つの確率分布の遠さを表す，距離のような量と見なすことができるが，上記の2つ目のことから分かるように，距離としての性質は満たさない（対称性を満たさない）ことに注意が必要である．

#### EM アルゴリズム

上述の式変形の結果を利用して対数尤度 $\log p_{\mathbf{\theta}}(\mathbf{x})$ を最大化するためには，元々のパラメータ $\mathbf{\theta}$ に加えて $q(z)$ もパラメータとして最適化する必要がある．
しかし，この最適化は，「$\mathbf{\theta}$を固定して $q(z)$ を更新する」手続き（「E-step」と呼ばれる）と，「$q(z)$を固定して$\mathbf{\theta}$を更新する」手続き（M-step）を交互に繰り返すことで実現できることが知られている．

- E-step では，$\mathbf{\theta}$ が固定されているので，$q(z)$ をどのように選んでも $\log p_{\mathbf{\theta}}(\mathbf{x})$ は変化しない．このとき， 新しい $q(z)$ を $q(z) = p_{\mathbf{\theta}}(z|\mathbf{x})$ ととれば， $D_{\rm KL}(q(z) \Vert p_{\mathbf{\theta}}(z|\mathbf{x})) = 0 $ となって ${\rm ELBO}(\mathbf{x}; \mathbf{\theta}, q)$ が最大となる．
- M-step では，$q(z)$ を上記で求めた値に固定して，${\rm ELBO}(\mathbf{x}; \mathbf{\theta}, q)$ を最大にする $\mathbf{\theta}$ を求める．


E-step と M-step を交互に繰り返して対数尤度を最大化するこの最適化のアルゴリズムを，**EMアルゴリズム** (Expectation-Maximization Algorithm) という．


ここまでの説明では，一つのデータに対する対数尤度 $\log p_{\mathbf{\theta}}(\mathbf{x})$ の最大化を考えていた．しかし，実際には，複数のデータに対する対数尤度
$L(\mathbf{\theta}) =\sum_{n=1}^N \log p_{\mathbf{\theta}}(\mathbf{x}_n)$
を最大化したい．
この場合，EMアルゴリズムの手続きは次のようになる（導出は省略）．

1. モデルパラメータ $\mathbf{\theta}$ を初期化する．
1. E-step: 現在の $\mathbf{\theta}$ を用いて $q_1, q_2, \ldots, q_N$ を更新する．
1. M-step: 次の式を最大にする $\mathbf{\theta}$ を求める．
$$
\sum_{n=1}^N {\rm ELBO}(\mathbf{x}_n; \mathbf{\theta}, q_n)
$$
1.　 繰り返しの終了条件を満たしていなければ 2. へ戻る


終了条件は，「$L(\mathbf{\theta})$ の変化量が既定値以下になった」や「繰り返しが既定の回数に達した」等とするのが一般的．

EMアルゴリズムでは，パラメータの更新を繰り返すごとに対数尤度 $L(\mathbf{\theta})$ が単調に増加する．すなわち，
$t$ 回目の E-step, M-step によって得られたパラメータを $\mathbf{\theta}_{t}$ とおき，この値を用いて次の E-step, M-step を実行して得られるパラメータを $\mathbf{\theta}_{t+1}$ とおくと，
$L(\mathbf{\theta}_{t+1}) \geq L(\mathbf{\theta}_{t})$ が成り立つ．



EMアルゴリズムの概要を説明してきたが，実は，ここまで具体的な GMM の式は出てきていない．
ここまでの議論は，GMM に限らず，潜在変数を持つモデル一般に成り立つものである．


モデルを GMM に限定してより具体的なアルゴリズムを導出する過程の説明は省略する．
結論としては，次のような手続きとなる．

E-step: 現在のパラメータ $\mathbf{\theta}$ つまり $w_k, \mathbf{\mu}_k, \Sigma_k$ を用いて $q_n(z = k) = p_{\mathbf{\theta}}(z = k|\mathbf{x})$ を求める．
以下，簡単のため，$q_n(z = k)$ を $q_{n,k}$ と表記する．
$q_{n, k}$ は，$\mathbf{x}_n$ が $k$ 番目の正規分布に所属する確率（その正規分布から生成された確率）の推定値とみなせる．

$$
q_{n, k} = \frac{w_k {\cal N}(\mathbf{x}_n; \mathbf{\mu}_k, \Sigma_k)}{\sum_{j=1}^K w_j {\cal N}(\mathbf{x}_n; \mathbf{\mu}_j, \Sigma_j)}
$$


M-step: 求めた $q_{n, k}$ を用いて，$w_k, \mathbf{\mu}_k, \Sigma_k$ を更新する．

$$
\begin{aligned}
w_k^{\rm new} &= \frac{1}{N}\sum_{n=1}^N q_{n, k}\\
\mathbf{\mu}_k^{\rm new} &= \frac{\sum_{n=1}^{N} q_{n, k} \mathbf{x}_n}{\sum_{n=1}^{N} q_{n, k}} \\
\Sigma_k^{\rm new} &= \frac{\sum_{n=1}^{N} q_{n, k} (\mathbf{x}_n - \mathbf{\mu}_k)(\mathbf{x}_n - \mathbf{\mu}_k)^{\top}}{\sum_{n=1}^{N} q_{n, k}}
\end{aligned}
$$



#### 実験: 別の2次元データ

In [ ]:
# データの入手
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/2dim3class.csv')
X = df.drop(columns='label').to_numpy()

In [ ]:
# グラフを描く
fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1])
xmin, xmax = -5, 5
ymin, ymax = -5, 5
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
plt.show()

In [ ]:
K = 3
gmm = GaussianMixture(n_components=K, covariance_type='full')
gmm.fit(X)

for k in range(K):
    print(f'### {k}-th component')
    print('weight = ', gmm.weights_[k])
    print('mu = ', gmm.means_[k])
    print('cov = ')
    print(gmm.covariances_[k])
    print()

In [ ]:
# グラフ描画用のグリッドデータの作成
x_mesh, y_mesh = np.mgrid[xmin:xmax:(xmax-xmin)/100, ymin:ymax:(ymax-ymin)/100]
X_mesh = np.dstack((x_mesh, y_mesh))

# グラフ
fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1])

# Gaussian を当てはめた結果
for k in range(K):
    mu = gmm.means_[k]
    cov = gmm.covariances_[k]
    ax.contour(x_mesh, y_mesh, multivariate_normal.pdf(X_mesh, mean=mu, cov=cov))
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')

[sklearn.mixture.GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) を読んで，`K` や `covariance_type` を変えて実験してみよう．